## Importar Librerias

In [1]:
# modulos de python
import ast

# modulos de terceros
import pandas as pd



> Importacion solo de las librerias a utilizar durante el ETL de los datos.

## Cargar datos.

In [2]:
# cargar datos de los creditos
path = '../data/credits.csv'
data_cd = pd.read_csv(path)
data_cd

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [3]:
data_cd['cast'][1]

"[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a36847f80a7c73', 'gender': 2, 'id': 2157, 'name': 'Robin Williams', 'order': 0, 'profile_path': '/sojtJyIV3lkUeThD7A2oHNm8183.jpg'}, {'cast_id': 8, 'character': 'Samuel Alan Parrish / Van Pelt', 'credit_id': '52fe44bfc3a36847f80a7c99', 'gender': 2, 'id': 8537, 'name': 'Jonathan Hyde', 'order': 1, 'profile_path': '/7il5D76vx6QVRVlpVvBPEC40MBi.jpg'}, {'cast_id': 2, 'character': 'Judy Sheperd', 'credit_id': '52fe44bfc3a36847f80a7c77', 'gender': 1, 'id': 205, 'name': 'Kirsten Dunst', 'order': 2, 'profile_path': '/wBXvh6PJd0IUVNpvatPC1kzuHtm.jpg'}, {'cast_id': 24, 'character': 'Peter Shepherd', 'credit_id': '52fe44c0c3a36847f80a7ce7', 'gender': 0, 'id': 145151, 'name': 'Bradley Pierce', 'order': 3, 'profile_path': '/j6iW0vVA23GQniAPSYI6mi4hiEW.jpg'}, {'cast_id': 10, 'character': 'Sarah Whittle', 'credit_id': '52fe44bfc3a36847f80a7c9d', 'gender': 1, 'id': 5149, 'name': 'Bonnie Hunt', 'order': 4, 'profile_path': '/7spiVQwmr

> carga de los datos de 'credits' y se evalua su informacion para poder proceder con las transformaciones.

In [4]:
# Evaluar los datos de todas las columnas.
data_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


## Cargar los datos de 'movies'

In [5]:
# Cargar los datos de las Peliculas.
path = '../data/movies_dataset.csv'
data_movies = pd.read_csv(path, low_memory=False)
data_movies

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


> Una vez cargado los datos se realiza una evaluacion e interpretacion para proceder con la limpueza y estructuracion del conjunto de datos (ETL)

In [6]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

## Transformar columna 'id'

In [7]:
# Transformar los datos de tipo int a str
data_cd['id'] = data_cd['id'].astype(str)

> Se tomo la decision de hacer un merge de ambos data sets con el fin de facilitar navegacion relacional de los datos entre las columnas, para ello era requerida la transformacion de la columna 'id' en los datos de 'credits' para poder manejar el mismo tipo de datos en ambas columnas y pueda hacerse un merge de ambos dataframes.

## Combinar Dataframes

In [8]:
# combinar datasets
data = pd.merge(data_movies, data_cd, on='id', how='inner')

In [9]:
# Verificar data combinada
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de..."
45534,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45535,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de..."
45536,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de..."


> una vez combinadas las tablas verificamos como ha quedado, analizando la cantidad de filas, columnas, tipo de las mismas, filas, datos nulos para proceder con una limpieza mas profunda.

In [10]:
# identificar valores nulos de cada tabla por orden descendiente.
data.isnull().sum().sort_values(ascending=False)

belongs_to_collection    41038
homepage                 37746
tagline                  25099
overview                   954
poster_path                386
runtime                    260
release_date                87
status                      84
imdb_id                     17
original_language           11
spoken_languages             3
revenue                      3
title                        3
video                        3
vote_average                 3
production_companies         3
production_countries         3
popularity                   3
vote_count                   3
original_title               0
genres                       0
id                           0
adult                        0
budget                       0
cast                         0
crew                         0
dtype: int64

## Verificar y eliminar valores duplicados.

In [11]:
# Verificar los datos duplicados del dataset.
data_d = data[data.duplicated()]

data_d

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
677,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,105045,tt0111613,de,Das Versprechen,"East-Berlin, 1961, shortly after the erection ...",...,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,False,5.0,1.0,"[{'cast_id': 5, 'character': 'Sophie II', 'cre...","[{'credit_id': '52fe4a44c3a36847f81c463f', 'de..."
952,False,NaN,4,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,22649,tt0022879,en,A Farewell to Arms,British nurse Catherine Barkley (Helen Hayes) ...,...,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every woman who has loved will understand,A Farewell to Arms,False,6.2,29.0,"[{'cast_id': 1, 'character': 'Catherine Barkle...","[{'credit_id': '56d73d3e9251414291002436', 'de..."
1468,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,105045,tt0111613,de,Das Versprechen,"East-Berlin, 1961, shortly after the erection ...",...,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,False,5.0,1.0,"[{'cast_id': 5, 'character': 'Sophie II', 'cre...","[{'credit_id': '52fe4a44c3a36847f81c463f', 'de..."
1469,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,105045,tt0111613,de,Das Versprechen,"East-Berlin, 1961, shortly after the erection ...",...,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,False,5.0,1.0,"[{'cast_id': 5, 'character': 'Sophie II', 'cre...","[{'credit_id': '52fe4a44c3a36847f81c463f', 'de..."
2569,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,84198,tt1736049,en,A Place at the Table,"Using personal stories, this powerful document...",...,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,One Nation. Underfed.,A Place at the Table,False,6.9,7.0,"[{'cast_id': 3, 'character': 'Himself', 'credi...","[{'credit_id': '52fe48e09251416c9109b347', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40346,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,97995,tt0127834,en,Seven Years Bad Luck,"After breaking a mirror in his home, superstit...",...,62.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Seven Years Bad Luck,False,5.6,4.0,"[{'cast_id': 1001, 'character': 'Max.', 'credi...","[{'credit_id': '55140dfa92514103e9002d99', 'de..."
44891,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144.0,"[{'cast_id': 1, 'character': 'Ash Ketchum', 'c...","[{'credit_id': '52fe43de9251416c750213ff', 'de..."
44897,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82.0,"[{'cast_id': 1, 'character': 'Ash Ketchum (voi...","[{'credit_id': '52fe45049251416c75048e35', 'de..."
45336,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,265189,tt2121382,sv,Turist,"While holidaying in the French Alps, a Swedish...",...,118.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,NaN,Force Majeure,False,6.8,255.0,"[{'cast_id': 2, 'character': 'Ebba', 'credit_i...","[{'credit_id': '534fd1a80e0a267eb6000e32', 'de..."


> Al aplicarsele una mascara a todo el conjunto de datos el dataset arrojo 82 Valores Duplicados en total 

In [12]:
# Copia del dataframe para eliminar los duplicados.
data_ = data.copy()
data = data_.drop_duplicates()
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de..."
45534,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de..."
45535,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de..."
45536,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de..."


In [13]:
# Se Verificanuevamente el estado de los valores duplicados.
data_d = data[data.duplicated()]

data_d

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew


> Al aplicar nuevamente la mascara al dataframe este arrojo 0 datos duplicados.

## Extraer datos de los jsons anidados

In [14]:
# Definir una función para extraer los datos anidados en listas.
def extract_list_str(list_str):
    '''la funcion realiza un casteo de str dependiendo
    de la sintaxis al tipo de estructura que se reconozca 
    para manipularse y retornar datos de la pelicula como str
      '''
    # Manejo de errores.
    try:
        # condicion para verificar que la el contexto del str no sea para una lista vacia.
        if list_str != '[]':

            # Usar el modulo ast para castear la el tipo a la estructura deseada.
            list_ = ast.literal_eval(list_str)
            # unir valores de la lista de js mediante lista comprimida y join.
            return ', '.join([dic['name'] for dic in list_])
        else:
            # Retornar NoData si es una lista vacia.
            return 'NoData'
    except:
        # Retornar NoData si es un valor nulo o existe otro tipo de error.
        return 'NoData'


# Definir una función para extraer los datos anidados de .
def extract_json_str(collection):
    '''la funcion realiza un casteo de str dependiendo
    de la sintaxis al tipo de estructura que se reconozca 
    para manipularse y retornar la coleccion de la pelicula como str
      '''
    # Manejo de errores.
    try:
        # Usar el modulo ast para castear la el tipo a la estructura deseada.
        collection_dict = ast.literal_eval(collection)
        # Devolver el valor extraido del diccionario.
        return collection_dict['name']
    except:
        # Retornar NoData si es un valor nulo o existe otro tipo de error.
        return 'NoData'

# Definir una función para extraer los datos anidados de genres.
def extract_director(director):
    '''la funcion realiza un casteo de str dependiendo
    de la sintaxis al tipo de estructura que se reconozca 
    para manipularse y retornar datos de la pelicula como str
      '''
    # Manejo de errores.
    try:
        # condicion para verificar que la el contexto del str no sea para una lista vacia.
        if director != '[]':
            # Usar el modulo ast para castear la el tipo a la estructura deseada.
            list_ = ast.literal_eval(director)
            # For para iterar la lista y encontrar el director entre el crew.
            for i in list_:
                if i['job'] == 'Director':
                    return i['name']
                else:
                    continue
        else:
            # Retornar NoData si es una lista vacia.
            return 'NoData'
    except:
        # Retornar NoData si es un valor nulo o existe otro tipo de error.
        return 'NoData'

# Aplicar las funciones de extracción a las columnas correspondientes
data['genres_clean'] = data['genres'].apply(extract_list_str)
data['collection_clean'] = data['belongs_to_collection'].apply(extract_json_str)
data['production_companies_clean'] = data['production_companies'].apply(extract_list_str)
data['production_countries_clean'] = data['production_countries'].apply(extract_list_str)
data['spoken_languages_clean'] = data['spoken_languages'].apply(extract_list_str)
data['cast_clean'] = data['cast'].apply(extract_list_str)
data['crew_clean'] = data['crew'].apply(extract_director)
# Mostrar las primeras filas del dataframe con las nuevas columnas
data

C:\Users\Rhamer\AppData\Local\Temp\ipykernel_13540\2204049089.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres_clean'] = data['genres'].apply(extract_list_str)
C:\Users\Rhamer\AppData\Local\Temp\ipykernel_13540\2204049089.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['collection_clean'] = data['belongs_to_collection'].apply(extract_json_str)
C:\Users\Rhamer\AppData\Local\Temp\ipykernel_13540\2204049089.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,vote_count,cast,crew,genres_clean,collection_clean,production_companies_clean,production_countries_clean,spoken_languages_clean,cast_clean,crew_clean
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","Animation, Comedy, Family",Toy Story Collection,Pixar Animation Studios,United States of America,English,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","Adventure, Fantasy, Family",NoData,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","Romance, Comedy",Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America,English,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","Comedy, Drama, Romance",NoData,Twentieth Century Fox Film Corporation,United States of America,English,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",Comedy,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America,English,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,1.0,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...","Drama, Family",NoData,NoData,Iran,فارسی,"Leila Hatami, Kourosh Tahami, Elham Korda",Hamid Nematollah
45534,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,3.0,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",Drama,NoData,Sine Olivia,Philippines,,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...",Lav Diaz
45535,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,6.0,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...","Action, Drama, Thriller",NoData,American World Pictures,United States of America,English,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...",Mark L. Lester
45536,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuy

> En este punto tras la evaluacion de los datos se puedo observar que los datos aninados tienen un formato de tipo cadena de texto por lo que requieren un tratamiento previo a la extraccion de los mismos. para ello se ha decidido a crear tres funcionen que interactuen con las filas mediante el metodo 'apply()' que trae consigo la libreria de pandas, estas funciones estan estructurada de la siguiente manera: <br> Las funciones de extract: <br>
recibe el parametro de la columna iterada e inicia con un 'Try-Except' para el manejo de Errores en dado caso de encontrar un valor nulo o cualquier otro error que pueda presentarse, dando como resultado que en consecuencia guardaria un 'NoData' de presentarse uno, caso contrario se correria un bloque de codigo donde se validaria mediante un 'if' que el str recibido no represente una lista vacia, caso contrario con el metodo 'literal_eval()' de la libreria 'ast'  que analiza la sintaxis de un string se hace un casteo automatico segun el texto, para luego poder demanera mas sencilla manipular el dato como una lista o un diccionario iterandolo o extrayendo directamente los datos.

## Eliminar columnas

In [15]:
# Columnas a eliminar.
drop_colums = ['video','poster_path', 'homepage','imdb_id','adult',
               'tagline','original_language', 'belongs_to_collection','genres',
               'production_companies','production_countries','spoken_languages',
                'cast', 'crew'
            ]
# Eliminar columnas con drop de pandas.
data = data.drop(columns=drop_colums)
data

,budget,id,original_title,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,genres_clean,collection_clean,production_companies_clean,production_countries_clean,spoken_languages_clean,cast_clean,crew_clean
0,30000000,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,Toy Story,7.7,5415.0,"Animation, Comedy, Family",Toy Story Collection,Pixar Animation Studios,United States of America,English,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,65000000,8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Jumanji,6.9,2413.0,"Adventure, Fantasy, Family",NoData,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,0,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Grumpier Old Men,6.5,92.0,"Romance, Comedy",Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America,English,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,16000000,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,6.1,34.0,"Comedy, Drama, Romance",NoData,Twentieth Century Fox Film Corporation,United States of America,English,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,0,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,5.7,173.0,Comedy,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America,English,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45533,0,439050,رگ خواب,Rising and falling between a man and woman.,0.072051,NaN,0.0,90.0,Released,Subdue,4.0,1.0,"Drama, Family",NoData,NoData,Iran,فارسی,"Leila Hatami, Kourosh Tahami, Elham Korda",Hamid Nematollah
45534,0,111109,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,Century of Birthing,9.0,3.0,Drama,NoData,Sine Olivia,Philippines,,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...",Lav Diaz
45535,0,67758,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,Betrayal,3.8,6.0,"Action, Drama, Thriller",NoData,American World Pictures,United States of America,English,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...",Mark L. Lester
45536,0,227506,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,Satan Triumphant,0.0,0.0,NoData,NoData,Yermoliev,Russia,NoData,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...",Yakov Protazanov


> Con Drop de pandas se eliminaron las columnas que no sera de utilidad para las operaciones que se realizaran durante todo el proceso del proyecto

## Verificar como esta quedando el Dataset

In [16]:
# Ver columnas, valores y formato.
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45456 entries, 0 to 45537
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   budget                      45456 non-null  object 
 1   id                          45456 non-null  object 
 2   original_title              45456 non-null  object 
 3   overview                    44502 non-null  object 
 4   popularity                  45453 non-null  object 
 5   release_date                45369 non-null  object 
 6   revenue                     45453 non-null  float64
 7   runtime                     45196 non-null  float64
 8   status                      45372 non-null  object 
 9   title                       45453 non-null  object 
 10  vote_average                45453 non-null  float64
 11  vote_count                  45453 non-null  float64
 12  genres_clean                45456 non-null  object 
 13  collection_clean            45456 no

In [17]:
# Ver valores nulos actuales.
data.isnull().sum().sort_values(ascending=False)

overview                      954
runtime                       260
crew_clean                    116
release_date                   87
status                         84
title                           3
vote_count                      3
vote_average                    3
revenue                         3
popularity                      3
budget                          0
original_title                  0
id                              0
collection_clean                0
genres_clean                    0
production_companies_clean      0
production_countries_clean      0
spoken_languages_clean          0
cast_clean                      0
dtype: int64

## Llenar valores nulos de 'revenue' y 'budget'

In [18]:
# Reemplazar valores nulos con 0
data['revenue'] = data['revenue'].fillna(0)
data['revenue']

0        373554033.0
1        262797249.0
2                0.0
3         81452156.0
4         76578911.0
            ...     
45533            0.0
45534            0.0
45535            0.0
45536            0.0
45537            0.0
Name: revenue, Length: 45456, dtype: float64

In [19]:
# Verificar si existen valores nulos.
data['revenue'][data['revenue'].isnull()]

Series([], Name: revenue, dtype: float64)

In [20]:
# castear el tipo de dato a float
data['budget'] = data['budget'].astype('float64')
# Verificar el tipo de dato
type(data['budget'][1])

numpy.float64

> Con el fin de poder realizar un calculo entre las columnas de 'revenue' y 'budget' para obtener el retorno de inversion se procedio a convertir los datos a int, normalizar nulos.

## Crear columna 'retorno_de_inversion'

In [21]:
# Crear y calcular columna de retorno de inversion
data.loc[:, 'retorno_de_inversion'] = data.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)
# Verificar la collumna.
data['retorno_de_inversion']

0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45533     0.000000
45534     0.000000
45535     0.000000
45536     0.000000
45537     0.000000
Name: retorno_de_inversion, Length: 45456, dtype: float64

> Mediante la nueva columna se puede visualizar el ROI de las peliculas segun la inversion, exeptuando algunas donde no se tenian los datos completos. <br>
Para ello se utiliza una funcion 'lambda' en la cual se divide el valor tomado de la columna 'revenue' de la fila y el valor de la columna 'budget' de la misma fila y se divide el  valor de 'revenue' entre el valor de 'budget' tomando encuenta que hay algunos valores en 'budget' que tienen valor de cero y no se puede dividir por esta cifra, se condiciona con un 'if else' donde se indica que si el valor de 'budget' es 0 en ese resultado colocar directamente 0

## Eliminar valores nulos de 'release_date'

In [22]:
# Eliminar los datos vacios en la columna release_data.
data= data.dropna(subset=['release_date'])
data

,budget,id,original_title,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,genres_clean,collection_clean,production_companies_clean,production_countries_clean,spoken_languages_clean,cast_clean,crew_clean,retorno_de_inversion
0,30000000.0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,Toy Story,7.7,5415.0,"Animation, Comedy, Family",Toy Story Collection,Pixar Animation Studios,United States of America,English,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,12.451801
1,65000000.0,8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Jumanji,6.9,2413.0,"Adventure, Fantasy, Family",NoData,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,4.043035
2,0.0,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Grumpier Old Men,6.5,92.0,"Romance, Comedy",Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America,English,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,0.000000
3,16000000.0,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,6.1,34.0,"Comedy, Drama, Romance",NoData,Twentieth Century Fox Film Corporation,United States of America,English,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,5.090760
4,0.0,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,5.7,173.0,Comedy,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America,English,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45532,0.0,30840,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,Robin Hood,5.7,26.0,"Drama, Action, Romance",NoData,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States...",English,"Patrick Bergin, Uma Thurman, David Morrissey, ...",John Irvin,0.000000
45534,0.0,111109,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,Century of Birthing,9.0,3.0,Drama,NoData,Sine Olivia,Philippines,,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...",Lav Diaz,0.000000
45535,0.0,67758,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,Betrayal,3.8,6.0,"Action, Drama, Thriller",NoData,American World Pictures,United States of America,English,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...",Mark L. Lester,0.000000
45536,0.0,227506,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,Satan Triumphant,0.0,0.0,NoData,NoData,Yermoliev,Russia,NoData,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...",Yakov Protazanov,0.000000


In [23]:
# Verificar valores nulos en las columnas procesadas.
data.isnull().sum().sort_values(ascending=False)

overview                      941
runtime                       246
crew_clean                    112
status                         80
original_title                  0
id                              0
budget                          0
release_date                    0
revenue                         0
title                           0
vote_average                    0
popularity                      0
vote_count                      0
genres_clean                    0
production_companies_clean      0
collection_clean                0
production_countries_clean      0
spoken_languages_clean          0
cast_clean                      0
retorno_de_inversion            0
dtype: int64

## Normalizar fecha

In [24]:
# Evaluar la columna 'release_date'
data['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45532    1991-05-13
45534    2011-11-17
45535    2003-08-01
45536    1917-10-21
45537    2017-06-09
Name: release_date, Length: 45369, dtype: object

In [25]:
# Cambiar el formato de la columna fecha a tipo date.
data.release_date = pd.to_datetime(data['release_date'], format='%Y-%m-%d', errors='coerce')
data.loc[:, 'release_date'] = pd.to_datetime(data['release_date'], format='%Y-%m-%d', errors='coerce')
data['release_date']

C:\Users\Rhamer\AppData\Local\Temp\ipykernel_13540\2342812240.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.release_date = pd.to_datetime(data['release_date'], format='%Y-%m-%d', errors='coerce')


0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45532   1991-05-13
45534   2011-11-17
45535   2003-08-01
45536   1917-10-21
45537   2017-06-09
Name: release_date, Length: 45369, dtype: datetime64[ns]

> Para que se pueda guardar correctamente el formato de la fecha y la hora en este punto se procedio a elaborar primero la transformacion con una imagen y luego se procedio a guardarla con 'loc' debido a que pandas nos enviaba un mensaje de advertencia sobre ello y se guardaba con el formato YY-mm-dd hh:mm:ss

## Crear columna 'release_year'

In [26]:
# Crear columna con el año del lanzamiento de la pelicula
data.loc[:, 'release_year'] = data['release_date'].dt.year

C:\Users\Rhamer\AppData\Local\Temp\ipykernel_13540\1520776411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'release_year'] = data['release_date'].dt.year


In [27]:
# Verificar la columna release_year.
data['release_year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45532    1991
45534    2011
45535    2003
45536    1917
45537    2017
Name: release_year, Length: 45369, dtype: int32

In [28]:
data.isnull().sum().sort_values(ascending=False)

overview                      941
runtime                       246
crew_clean                    112
status                         80
budget                          0
popularity                      0
original_title                  0
id                              0
revenue                         0
title                           0
vote_average                    0
vote_count                      0
release_date                    0
genres_clean                    0
collection_clean                0
production_countries_clean      0
production_companies_clean      0
spoken_languages_clean          0
cast_clean                      0
retorno_de_inversion            0
release_year                    0
dtype: int64

In [29]:
data = data.dropna()

In [30]:
# Dataframe completo.
data

,budget,id,original_title,overview,popularity,release_date,revenue,runtime,status,title,...,vote_count,genres_clean,collection_clean,production_companies_clean,production_countries_clean,spoken_languages_clean,cast_clean,crew_clean,retorno_de_inversion,release_year
0,30000000.0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,Toy Story,...,5415.0,"Animation, Comedy, Family",Toy Story Collection,Pixar Animation Studios,United States of America,English,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter,12.451801,1995
1,65000000.0,8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Jumanji,...,2413.0,"Adventure, Fantasy, Family",NoData,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"English, Français","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston,4.043035,1995
2,0.0,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Grumpier Old Men,...,92.0,"Romance, Comedy",Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America,English,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch,0.000000,1995
3,16000000.0,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,...,34.0,"Comedy, Drama, Romance",NoData,Twentieth Century Fox Film Corporation,United States of America,English,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker,5.090760,1995
4,0.0,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,...,173.0,Comedy,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America,English,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer,0.000000,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45532,0.0,30840,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,Robin Hood,...,26.0,"Drama, Action, Romance",NoData,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States...",English,"Patrick Bergin, Uma Thurman, David Morrissey, ...",John Irvin,0.000000,1991
45534,0.0,111109,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,Century of Birthing,...,3.0,Drama,NoData,Sine Olivia,Philippines,,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...",Lav Diaz,0.000000,2011
45535,0.0,67758,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,Betrayal,...,6.0,"Action, Drama, Thriller",NoData,American World Pictures,United States of America,English,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...",Mark L. Lester,0.000000,2003
45536,0.0,227506,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,Satan Triumphant,...,0.0,NoData,NoData,Yermoliev,Russia,NoData,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...",Yakov Protazanov,0.000000,1917


<p>Con esto se daria por terminado el proceso de ETL de todo el conjunto de datos hasta este punto. quedando un con junto de datos de 44264 filas 21 </p>

In [34]:
data2 = pd.read_csv('data.csv')

In [35]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44264 entries, 0 to 44263
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   budget                      44264 non-null  float64
 1   id                          44264 non-null  int64  
 2   original_title              44264 non-null  object 
 3   overview                    44264 non-null  object 
 4   popularity                  44264 non-null  float64
 5   release_date                44264 non-null  object 
 6   revenue                     44264 non-null  float64
 7   runtime                     44264 non-null  float64
 8   status                      44264 non-null  object 
 9   title                       44264 non-null  object 
 10  vote_average                44264 non-null  float64
 11  vote_count                  44264 non-null  float64
 12  genres_clean                44264 non-null  object 
 13  collection_clean            442